In [1]:
import pandas as pd
import numpy as np
# fast ai is used only for easy data transformations
from fastai import *
from fastai.tabular import *
from fastai.tabular.all import *
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [2]:
train_df=pd.read_csv("../sample_data/train.csv",low_memory=False)
test_df=pd.read_csv("../sample_data/test.csv",low_memory=False)

In [3]:
# Remove rows where all elements are NaN
org_train_len=len(train_df)
train_df = train_df.dropna(how='all')
clean_train_len=len(train_df)
org_test_len=len(test_df)
test_df = test_df.dropna(how='all')
clean_test_len=len(test_df)

In [4]:
train_df['date'] = pd.to_datetime(train_df['date'], errors='coerce')
# Count the number of NaN values in the date column
nan_count = train_df['date'].isna().sum()
len(train_df),nan_count

/var/folders/1m/ps1kwqdn5p1bdj1943362s500000gq/T/ipykernel_80412/1682223351.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_df['date'] = pd.to_datetime(train_df['date'], errors='coerce')


(6480, 0)

In [5]:
# Define a conversion function with NaN handling
def convert_to_liters(value):
    # Check for NaN (or similar) values
    if pd.isna(value):
        # Choose your strategy here: return None, or a default value, or raise an error, etc.
        return np.nan  # Here, we choose to return NaN as is
    # Split the number and unit
    number, unit = float(value[:-2]), value[-2:]
    # Convert to liters if necessary
    if unit == 'ml':
        return number / 1000  # Convert ml to l
    elif unit == 'lt':
        return number
    else:
        raise ValueError(f"Unknown unit: {unit}")

In [6]:
# Apply the conversion to the entire column
train_df['capacity_liters'] = train_df['capacity'].apply(convert_to_liters)
test_df['capacity_liters'] = test_df['capacity'].apply(convert_to_liters)

In [7]:
train_df = train_df.drop(columns=['capacity'])
test_df = test_df.drop(columns=['capacity'])

In [8]:
add_datepart(train_df, 'date')
add_datepart(test_df,'date')

/opt/anaconda3/envs/ML_OPS/lib/python3.10/site-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)
/opt/anaconda3/envs/ML_OPS/lib/python3.10/site-packages/fastai/tabular/core.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)


,id,city,lat,long,pop,shop,brand,container,price,quantity,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,6480,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,plastic,3.10,7056,...,31,2,31,True,False,False,False,False,False,1.517357e+09
1,6481,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,can,0.85,12490,...,31,2,31,True,False,False,False,False,False,1.517357e+09
2,6482,Athens,37.97945,23.71622,664046,shop_1,adult-cola,glass,0.83,26640,...,31,2,31,True,False,False,False,False,False,1.517357e+09
3,6483,Athens,37.97945,23.71622,664046,shop_1,orange-power,glass,0.54,41892,...,31,2,31,True,False,False,False,False,False,1.517357e+09
4,6484,Athens,37.97945,23.71622,664046,shop_1,orange-power,plastic,0.83,22923,...,31,2,31,True,False,False,False,False,False,1.517357e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,7555,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,plastic,2.52,13760,...,31,0,365,True,False,True,False,True,False,1.546214e+09
1076,7556,Athens,37.97945,23.71622,664046,shop_1,orange-power,plastic,2.18,16309,...,31,0,365,True,False,True,False,True,False,1.546214e+09
1077,7557,Patra,38.24444,21.73444,168034,shop_6,kinder-cola,can,0.85,24378,...,31,0,365,True,False,True,False,True,False,1.546214e+09
1078,7558,Thessaloniki,40.64361,22.93086,354290,shop_4,adult-cola,plastic,2.17,20691,...,31,0,365,True,False,True,False,True,False,1.546214e+09


In [9]:
train_df['quantity'] = np.log(train_df['quantity'])
test_df['quantity'] = np.log(test_df['quantity'])

In [10]:
cat_names = ['city', 'shop', 'brand', 'container']
cont_names = ['lat', 'long', 'pop','capacity_liters','price']
procs = [Categorify, FillMissing, Normalize]
y_names = 'quantity'

In [ ]:
train_df

In [16]:

tdf_train = TabularPandas(train_df, procs=procs, cat_names=cat_names, cont_names=cont_names,y_block=RegressionBlock(),
                   y_names=y_names)
tdf_test = TabularPandas(test_df, procs=procs, cat_names=cat_names, cont_names=cont_names,y_block=RegressionBlock(),
                   y_names=y_names)

In [17]:
df_train = tdf_train.items
df_test = tdf_test.items

In [18]:
df_test

,id,city,lat,long,pop,shop,brand,container,price,quantity,...,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,lat_na,long_na,capacity_liters_na
0,6480,1,-0.193103,0.416100,1.341481,1,3,3,2.245355,8.861633,...,True,False,False,False,False,False,1.517357e+09,1,1,1
1,6481,1,-0.193103,0.416100,1.341481,1,3,1,-0.468544,9.432684,...,True,False,False,False,False,False,1.517357e+09,1,1,1
2,6482,1,-0.193103,0.416100,1.341481,1,1,2,-0.492668,10.190169,...,True,False,False,False,False,False,1.517357e+09,1,1,1
3,6483,1,-0.193103,0.416100,1.341481,1,5,2,-0.842459,10.642850,...,True,False,False,False,False,False,1.517357e+09,1,1,1
4,6484,1,-0.193103,0.416100,1.341481,1,5,3,-0.492668,10.039896,...,True,False,False,False,False,False,1.517357e+09,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,7555,1,-0.193103,0.416100,1.341481,1,3,3,1.545772,9.529521,...,True,False,True,False,True,False,1.546214e+09,1,1,1
1076,7556,1,-0.193103,0.416100,1.341481,1,5,3,1.135672,9.699472,...,True,False,True,False,True,False,1.546214e+09,1,1,1
1077,7557,4,-0.032382,-1.414598,-0.814148,6,3,1,-0.468544,10.101437,...,True,False,True,False,True,False,1.546214e+09,1,1,1
1078,7558,5,1.422763,-0.309388,-0.004694,4,1,3,1.123610,9.937454,...,True,False,True,False,True,False,1.546214e+09,1,1,1
